# RDDs

## Creamos un contexo para crear RDDs

In [1]:
from pyspark import SparkContext
sc = SparkContext(master = "local", appName = "Transformaciones sobre un RDD")

21/12/16 14:39:10 WARN Utils: Your hostname, localhost resolves to a loopback address: 127.0.0.1; using 172.18.0.2 instead (on interface eth0)
21/12/16 14:39:10 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/12/16 14:39:11 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## Cargamos un RDDs

Para la realzación del ejercicio, con ayuda del archivo paises, realizaremos el equivalente de operaciones 'select','count','group by' y 'filter / where'

Cambia el valor de la ruta para que apunte a la ruta donde tienes los datos

In [2]:
path = "data/"
equiposOlimpicosRDD = sc.textFile(path+"paises.csv").map(lambda line : line.split(","))

In [3]:
equiposOlimpicosRDD.take(10)

[['id', 'equipo', 'sigla'],
 ['1', '30. Februar', 'AUT'],
 ['2', 'A North American Team', 'MEX'],
 ['3', 'Acipactli', 'MEX'],
 ['4', 'Acturus', 'ARG'],
 ['5', 'Afghanistan', 'AFG'],
 ['6', 'Akatonbo', 'IRL'],
 ['7', 'Alain IV', 'SUI'],
 ['8', 'Albania', 'ALB'],
 ['9', 'Alcaid', 'POR']]

### Determinamos la cantidad de siglas para los equipos olímpicos existentes

Con ayuda del método 'take', podrás identificar que en la ultima posisión tenemos las siguas para cada pais/equipo. 

Con esto obtenremos la cantidad precisa de equipos jugadores en los olimpicos

In [4]:
equiposOlimpicosRDD.map(lambda x : (x[2])).distinct().count()

231

### Agrupamos datos para poder determinar cuantos equipos posee un pais

Recuerda que los RDDs poseen una estructura de llave-valor, por lo cual debemos poner primero el valor 'llave'.

Con 'mapValues', al componente 'valor' le indicamos que operación deseamos que se le aplique.

In [5]:
equiposOlimpicosRDD.map(lambda x : (x[2], x[1] )).groupByKey().mapValues(list).take(3)


[('sigla', ['equipo']),
 ('AUT',
  ['30. Februar',
   'Austria',
   'Austria-1',
   'Austria-2',
   'Breslau',
   'Brigantia',
   'Donar III',
   'Evita VI',
   'May-Be 1960',
   '"R.-V. Germania; Leitmeritz"',
   'Surprise']),
 ('MEX',
  ['A North American Team',
   'Acipactli',
   'Chamukina',
   'Mexico',
   'Mexico-1',
   'Mexico-2',
   'Nausikaa 4',
   'Tlaloc',
   'Xolotl'])]

In [6]:
equiposOlimpicosRDD.map(lambda x :  (x[2], x[1] )).groupByKey().mapValues(len).take(5)


[('sigla', 1), ('AUT', 11), ('MEX', 9), ('ARG', 18), ('AFG', 1)]

### Operación filter para obtener un subconjunto

Con el método 'filter', reducimos el conjuntos de equipos.

Nos quedamos con la sigla equivalente de argentina

In [7]:
equiposArgentinos = equiposOlimpicosRDD.filter(lambda l : "ARG" in l)
equiposArgentinos.collect()  

[['4', 'Acturus', 'ARG'],
 ['37', 'Antares', 'ARG'],
 ['42', 'Arcturus', 'ARG'],
 ['43', 'Ardilla', 'ARG'],
 ['45', 'Argentina', 'ARG'],
 ['46', 'Argentina-1', 'ARG'],
 ['47', 'Argentina-2', 'ARG'],
 ['119', 'Blue Red', 'ARG'],
 ['238', 'Covunco III', 'ARG'],
 ['252', 'Cupidon III', 'ARG'],
 ['288', 'Djinn', 'ARG'],
 ['436', 'Gullvinge', 'ARG'],
 ['644', 'Matrero II', 'ARG'],
 ['672', 'Mizar', 'ARG'],
 ['774', 'Pampero', 'ARG'],
 ['843', 'Rampage', 'ARG'],
 ['1031', 'Tango', 'ARG'],
 ['1162', 'Wiking', 'ARG']]

## Accciones sobre RDDs

In [8]:
deportistaOlimpicoRDD = sc.textFile(path+"deportista.csv").map(lambda line : line.split(","))
deportistaOlimpico2RDD = sc.textFile(path+"deportista2.csv").map(lambda line : line.split(","))

In [23]:
deportistaOlimpicoRDD = deportistaOlimpicoRDD.union(deportistaOlimpico2RDD)
deportistaOlimpicoRDD.take(5)

[['deportista_id', 'nombre', 'genero', 'edad', 'altura', 'peso', 'equipo_id'],
 ['1', 'A Dijiang', '1', '24', '180', '80', '199'],
 ['2', 'A Lamusi', '1', '23', '170', '60', '199'],
 ['3', 'Gunnar Nielsen Aaby', '1', '24', '0', '0', '273'],
 ['4', 'Edgar Lindenau Aabye', '1', '34', '0', '0', '278']]

### Formas de visualizar datos de un RDDs

La operación 'take' nos devuelve 'N' valores que encuentre spark.

La operación 'top', previo ordena respecto al valor llave y nos devuelve 'N' valores.

La operacion 'takeSample', nos devuelve una muestra aleatoria de los valores, Observa que recibe tres parametros

| Orden | Argumento | Descripción | Valor
|-------|--------|-----|--------|
|1|withReplacement|Indica si la muetra podrá traer replicados|Bool|
|2| num| Cantidad de valores a retornar|int|
|3|sed|semilla para el generador aleatorio|int|

Nota: Si encuentras complicado leer el código en los segmentos donde usamos indices en las listas, ejecutalo por partes para que visualizes que componentes seleccionamos 

Nota: Para hacer join con RDDs, debemos tener selecionada la llave al inicio del RDD para poder realizar el cruce.

In [10]:
deportistaOlimpicoRDD.map(lambda x :[x[-1], x[:-1]]).join(equiposOlimpicosRDD \
                        .map(lambda x : [x[0],x[2]])).take(6)

[('199', (['1', 'A Dijiang', '1', '24', '180', '80'], 'CHN')),
 ('199', (['2', 'A Lamusi', '1', '23', '170', '60'], 'CHN')),
 ('199', (['602', 'Abudoureheman', '1', '22', '182', '75'], 'CHN')),
 ('199', (['1463', 'Ai Linuer', '1', '25', '160', '62'], 'CHN')),
 ('199', (['1464', 'Ai Yanhan', '2', '14', '168', '54'], 'CHN')),
 ('199', (['3605', 'An Weijiang', '1', '22', '178', '72'], 'CHN'))]

In [11]:
deportistaOlimpicoRDD.map(lambda x :[x[-1], x[:-1]]).join(equiposOlimpicosRDD \
                        .map(lambda x : [x[0],x[2]])).top(5)

[('999', (['92679', 'Trygve Bjarne Pedersen', '1', '35', '0', '0'], 'NOR')),
 ('999', (['1144', 'Henrik Agersborg', '1', '47', '0', '0'], 'NOR')),
 ('999', (['10765', 'Einar Berntsen', '1', '28', '0', '0'], 'NOR')),
 ('998',
  (['111659', 'G. Bernard Bernie Skinner', '1', '34', '182', '82'], 'CAN')),
 ('996', (['116030', 'Edward Eddy Stutterheim', '1', '39', '0', '0'], 'NED'))]

In [12]:
deportistaOlimpicoRDD.map(lambda x :[x[-1], x[:-1]]).join(equiposOlimpicosRDD \
                        .map(lambda x : [x[0],x[2]])).takeSample(False,4,10)

[('93',
  (['76502', 'Yelena Viktorovna Matoshko', '2', '30', '177', '80'], 'BLR')),
 ('487', (['110997', 'Ajit Singh', '1', '23', '185', '73'], 'IND')),
 ('249',
  (['41654', 'Toms Pedro Gonzlez Barrios', '1', '21', '178', '75'], 'CUB')),
 ('259', (['7157', 'Jlius Bal', '1', '22', '0', '0'], 'TCH'))]

### Importancia de countAprox

Debido a la cantidad de datos no siempre es recomendable hacer operaciones tipo count.

Por lo que 'countAprox' es la solución mas viable cuando solo queremos darnos una idea de cuantos datos podemos leer durante un tiempo determinado. Nota: el parametro está en milisegundos

In [13]:
deportistaOlimpicoRDD.map(lambda x :[x[-1], x[:-1]]).join(equiposOlimpicosRDD \
                            .map(lambda x : [x[0],x[2]])).count()

135427

In [14]:
deportistaOlimpicoRDD.map(lambda x :[x[-1], x[:-1]]).join(equiposOlimpicosRDD \
                        .map(lambda x : [x[0],x[2]])).countApprox(20)

135427

## Acciones de modificacion

### Obtenemos el equipo y el deportista

Guardamos el RDD resultante en equipoDeportista

In [15]:
equipoDeportista = deportistaOlimpicoRDD.map(lambda x :[x[-1], x[:-1]]).join(equiposOlimpicosRDD \
                                            .map(lambda x : [x[0],x[2]]))

In [25]:
equipoDeportista.take(5)

[('199', (['1', 'A Dijiang', '1', '24', '180', '80'], 'CHN')),
 ('199', (['2', 'A Lamusi', '1', '23', '170', '60'], 'CHN')),
 ('199', (['602', 'Abudoureheman', '1', '22', '182', '75'], 'CHN')),
 ('199', (['1463', 'Ai Linuer', '1', '25', '160', '62'], 'CHN')),
 ('199', (['1464', 'Ai Yanhan', '2', '14', '168', '54'], 'CHN'))]

In [30]:
equipoDeportista.map(lambda x : (x[1][0][0],x[1][0][1:],x[1][1],x[0])).take(5)

[('1', ['A Dijiang', '1', '24', '180', '80'], 'CHN', '199'),
 ('2', ['A Lamusi', '1', '23', '170', '60'], 'CHN', '199'),
 ('602', ['Abudoureheman', '1', '22', '182', '75'], 'CHN', '199'),
 ('1463', ['Ai Linuer', '1', '25', '160', '62'], 'CHN', '199'),
 ('1464', ['Ai Yanhan', '2', '14', '168', '54'], 'CHN', '199')]

### Carga de resultados

Esta tabla posee las medallas que los jugadores han ganado

In [18]:
resultado = sc.textFile(path+"resultados.csv").map(lambda line : line.split(","))

Eliminamos todas las medallas no ganadoras y nos quedamos con el valor de la medalla y  del deportista_id

In [20]:
resultadoGanador = resultado.filter(lambda l : 'NA' not in l[1])
resultadoGanador.take(5)

[['resultado_id', 'medalla', 'deportista_id', 'juego_id', 'evento_id'],
 ['4', 'Gold', '4', '2', '4'],
 ['38', 'Bronze', '15', '7', '19'],
 ['39', 'Bronze', '15', '7', '20'],
 ['41', 'Bronze', '16', '50', '14']]

In [21]:
resultadoGanador = resultadoGanador.map(lambda l : [l[2],l[1]])
resultadoGanador.take(5)

[['deportista_id', 'medalla'],
 ['4', 'Gold'],
 ['15', 'Bronze'],
 ['15', 'Bronze'],
 ['16', 'Bronze']]

In [34]:
resultadoGanador.map(lambda l: [l[0],l[1]]).join( \
    equipoDeportista.map(lambda x: [x[1][0][0],x[1][0][1:],x[1][1],x[0]] )).take(5)

[('4', ('Gold', ['Edgar Lindenau Aabye', '1', '34', '0', '0'])),
 ('16', ('Bronze', ['Juhamatti Tapio Aaltonen', '1', '28', '184', '85'])),
 ('20', ('Gold', ['Kjetil Andr Aamodt', '1', '20', '176', '85'])),
 ('20', ('Bronze', ['Kjetil Andr Aamodt', '1', '20', '176', '85'])),
 ('20', ('Silver', ['Kjetil Andr Aamodt', '1', '20', '176', '85']))]

### Obtenemos la relación buscada: deportista,pais y medalla.

In [35]:
jugadoresMedalla =  equipoDeportista.join(resultadoGanador)
jugadoresMedalla.take(1)

[('716',
  ((['553', 'John Charles Abrams', '1', '22', '183', '0'], 'NZL'), 'Gold'))]

### Agrupamos las medallas respecto a la sigla del pais jugador

In [39]:
val_Medallas = {'Gold':7, 'Silver':5, 'Bronze':4}
paisesMedallas = jugadoresMedalla.map(lambda x : (x[1][0][-1], val_Medallas[x[1][1]]) )
paisesMedallas.takeSample(False,10)

[('ZAM', 7),
 ('TCH', 4),
 ('TCH', 4),
 ('GER', 5),
 ('NED', 7),
 ('NED', 5),
 ('LIB', 4),
 ('ITA', 4),
 ('ITA', 5),
 ('GER', 5)]

### Obtenemos los valores de los puntuajes históricos de los paises jugadores

In [42]:
from operator import add
conclusion = paisesMedallas.reduceByKey(add).sortBy(lambda x : x[1],ascending=False)
conclusion.take(10)

[('ITA', 74920),
 ('NED', 65560),
 ('GER', 22323),
 ('JPN', 19950),
 ('NZL', 9220),
 ('TCH', 8160),
 ('BLR', 5012),
 ('RSA', 4735),
 ('TUR', 3965),
 ('GHA', 3430)]

## Estadística básica sobre los RDDs

Spark posee una suite integrada de forma natural para poder obtener estadísticas básicas.

In [43]:
conclusion.map(lambda l : l[1]).stats()

(count: 65, mean: 3735.6153846153843, stdev: 12505.143665022015, max: 74920.0, min: 5.0)

In [44]:
conclusion.map(lambda l : l [1]).mean()

3735.6153846153843

In [45]:
conclusion.map(lambda l : l [1]).sum()

242815

In [46]:
conclusion.map(lambda l : l [1]).histogram(10)

([5.0,
  7496.5,
  14988.0,
  22479.5,
  29971.0,
  37462.5,
  44954.0,
  52445.5,
  59937.0,
  67428.5,
  74920],
 [59, 2, 2, 0, 0, 0, 0, 0, 1, 1])

In [47]:
sc.stop()